Prepare the environment

In [23]:
! pip install azure-ai-ml 
! pip install azure-identity 
! pip install keras
! pip install tensorflow
! pip install pandas
! pip install opencv-python
! pip install Pillow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [24]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace
from azure.identity import DefaultAzureCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to workspace
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<AML_WORKSPACE_NAME>"

    ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

We could not find config.json in: . or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.


In [25]:
import os
from keras.models import load_model
import numpy as np
import pandas as pd
import glob
import cv2
from helper_functions import imgs_to_array, get_lcd

# Directory structure:
root = '../Test_case/'
data = root + 'test_data/'
results = root + 'results/'
if not os.path.exists(results):
    os.mkdir(results)
lcd_data = 'temp/'
if not os.path.exists(lcd_data):
    os.mkdir(lcd_data)

#Load CNN model trained on data pre-defined in the paper
model=load_model('../Dataset/best_model.h5')

# Preprocess image in into binarized single monitor lcd_data with a label file
fname = data+'image1.jpg'

preprocessed_img = get_lcd(fname)

#Save Systolic and diastolic contours with _SP.jpg and _DP.jpg suffix respectively            
tag=os.path.basename(fname).split('.')[0]        #Filename Tag for LCD lcd_data
w, h=preprocessed_img.shape
cv2.imwrite(lcd_data + tag+'_SP.jpg', preprocessed_img[0:int(h/2),0:w])
cv2.imwrite(lcd_data + tag+'_DP.jpg', preprocessed_img[int(h/2):h,0:w])

#Adding image data into labels file
label_lst =[]
label_lst.append(tag+'_SP.jpg')
label_lst.append(tag+'_DP.jpg')

frame_labels = pd.DataFrame({'filename': label_lst})

frame_labels.to_csv("temp/labels_BP.csv")   
print("Single monitor lcd_data and label files created!")

../Test_case/test_data/image1.jpg
BP 76 132 311 256 79616 1.21484375 76764.0
Single monitor lcd_data and label files created!


In [28]:
#Transcription of all images in test folder
X_test = imgs_to_array(frame_labels.filename[:], lcd_data)
y_pred = model.predict(X_test)

# Predict Systolic and Diastolic BP values for the test image
for i in range(X_test.shape[0]):
    pred_list_i = [np.argmax(pred[i]) for pred in y_pred]
    predicted_num = 100* pred_list_i[0] + 10 * pred_list_i[1] + 1* pred_list_i[2]
    if predicted_num >= 1000:
        predicted_num = predicted_num-1000

    fname = frame_labels.filename[i]         
    if fname.endswith('_SP.jpg'):
        print("Systolic pressure: " + str(predicted_num))
    elif fname.endswith('_DP.jpg'):
        print("Diastolic pressure: " + str(predicted_num))

80 180
80 180
1/1 [==============================] - 0s 37ms/step
Systolic pressure: 105
Diastolic pressure: 72
